In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import unicodedata
import string
import os
from tqdm.auto import tqdm
import io
import glob
import random

In [ ]:
input_size = 28
sequence_length = 28
num_classes = 10
num_layers = 2
hidden_size = 256
learning_rate = 0.001
batch_size = 32
num_of_epochs = 5

* Input data shape = batch size x color channels x height x width

## Example of converting into sequential data

* source: https://www.geeksforgeeks.org/long-short-term-memory-networks-using-pytorch/

In [2]:
np.random.seed(0)
torch.manual_seed(0)

t = np.linspace(0, 100, 1000)
data = np.sin(t)

def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data)-seq_length):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, y = create_sequences(data, seq_length)

trainX = torch.tensor(X[:, :, None], dtype=torch.float32)
trainY = torch.tensor(y[:, None], dtype=torch.float32)

# Creating RNN Models

## RNN version 1

In [3]:
class RNNversion1(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNNversion1, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x: torch.Tensor):
       initial_hidden_state = torch.zeros(self.num_layers, x.size(0),self.hidden_size)
       x, _ = self.rnn(x, initial_hidden_state)
       x  = x[:, -1, :]
       x = self.fc(x)
       return x

## RNN Version 2

In [80]:
class RNNversion2(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNNversion2, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

    def forward(self, x: torch.Tensor):
        ##initialize hidden state
        initial_hidden_state = torch.zeros(self.num_layers, x.size(0),self.hidden_size)
        x, _ = self.rnn(x, initial_hidden_state)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x

## LSTM

In [4]:
class MyModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(MyModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Sequential(
            # nn.Flatten(start_dim=1),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )
    def forward(self, x: torch.Tensor, h0=None, c0=None):
        ##initialize hidden state
        if h0 is None or c0 is None:
            h0 = torch.zeros(self.num_layers, x.size(0),self.hidden_size)
            c0 = torch.zeros(self.num_layers, x.size(0),self.hidden_size)
        x, _ = self.lstm(x, (h0, c0))
        x = x[:, -1, :]
        x = self.fc(x)
        return x

In [26]:
# cnn_params = [16, 32, "MaxPool"]
cnn_params = [16, 16, "MaxPool"]
class CNN1D(nn.Module):
    def __init__(self, input):
        super(CNN1D, self).__init__()
        self.input = input
        self.conv_blocks = self.create_conv_blocks(cnn_params)
        
    def create_conv_blocks(self, architecture):
        layers = []
        input = self.input

        for x in architecture:
            if type(x) == int:
                output = x
                layers += [nn.Conv1d(input, output, 
                        kernel_size=2, stride=1, padding=0),
                        nn.BatchNorm1d(x),
                        nn.SiLU()]

                input = x  # Update input channels
            elif x == "MaxPool":
                layers += [nn.MaxPool1d(kernel_size=2, stride=1)]

        return nn.Sequential(*layers)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # (batch, features, time_steps) → (batch, channels, time_steps)
        x = self.conv_blocks(x)
        return x  # Output shape: (batch, channels, reduced_time_steps)

class MainModel(nn.Module):
    def __init__(self, input_size=16, hidden_size=100, num_layers=3, output_size=1):
        super(MainModel, self).__init__()
        self.num_layers = num_layers  
        self.hidden_size = hidden_size
        self.cnn = CNN1D(input_size)
        self.lstm = nn.LSTM(input_size=16, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, output_size), 
        )

    def forward(self, x: torch.Tensor, h0=None, c0=None):
        x = self.cnn(x)
        x = x.permute(0, 2, 1)  # (batch, channels, time) → (batch, time, channels)
        if h0 is None or c0 is None:
            h0 = torch.zeros(self.num_layers, x.size(0),self.hidden_size)
            c0 = torch.zeros(self.num_layers, x.size(0),self.hidden_size)
            # h0 = torch.randn(self.num_layers, x.size(0), self.hidden_size) * 0.02
            # c0 = torch.randn(self.num_layers, x.size(0), self.hidden_size) * 0.02
        x, _ = self.lstm(x, (h0, c0))
        x = x[:, -1, :]
        x = self.fc(x) 
        return x


### Mini training

In [29]:
# model = MyModel(input_size=1, hidden_size=100, num_layers=1, output_size=1)
model = MainModel(input_size=1, hidden_size=100, num_layers=1, output_size=1)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
num_epochs=100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass (h0, c0 are initialized inside the forward method if None)
    outputs = model(trainX)

    loss = criterion(outputs, trainY)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.7f}')


Epoch [10/100], Loss: 0.0136843
Epoch [20/100], Loss: 0.0022997
Epoch [30/100], Loss: 0.0018742
Epoch [40/100], Loss: 0.0004940
Epoch [50/100], Loss: 0.0001767
Epoch [60/100], Loss: 0.0000885
Epoch [70/100], Loss: 0.0000550
Epoch [80/100], Loss: 0.0000473
Epoch [90/100], Loss: 0.0000396
Epoch [100/100], Loss: 0.0000322


In [83]:
# Hyperparameters
input_size = 2
hidden_size = 100
num_layers = 20
num_classes = 50
sequence_length = 10
batch_size = 4

# Generate dummy data
# Input tensor of shape (batch_size, sequence_length, input_size)
dummy_input = torch.randn(batch_size, sequence_length, input_size)

# Instantiate the model
model1 = RNNversion1(input_size, hidden_size, num_layers, num_classes)
# model2 = RNNversion2(input_size, hidden_size, num_layers, num_classes)
model2 = LSTM(input_size, hidden_size, num_layers, num_classes)

# Forward pass with dummy data
output1 = model1(dummy_input)
output2 = model2(dummy_input)

# print("Input shape:", dummy_input.shape)
# print("Output shape:", output1.shape)
# print("Output:", output1)
print("Output shape:", output2.shape)
print("Output:", output2)

Output shape: torch.Size([4, 50])
Output: tensor([[ 0.0901,  0.1010,  0.0744,  0.0105,  0.1101,  0.1084, -0.0263, -0.0751,
         -0.0860,  0.0226,  0.0301, -0.0400, -0.0561, -0.0916, -0.0444, -0.0637,
          0.0251,  0.0408,  0.0745, -0.0650,  0.0215,  0.0650, -0.0265,  0.0188,
          0.0634,  0.0627,  0.0173,  0.0527,  0.0402, -0.0720,  0.0639, -0.0492,
          0.0640, -0.0485, -0.0953,  0.0530,  0.0525,  0.0348,  0.0398,  0.0923,
         -0.0609,  0.0202,  0.0518, -0.0108,  0.0164, -0.0496,  0.0081,  0.0815,
         -0.0600, -0.0083],
        [ 0.0901,  0.1010,  0.0744,  0.0105,  0.1101,  0.1084, -0.0263, -0.0751,
         -0.0860,  0.0226,  0.0301, -0.0400, -0.0561, -0.0916, -0.0444, -0.0637,
          0.0251,  0.0408,  0.0745, -0.0650,  0.0215,  0.0650, -0.0265,  0.0188,
          0.0634,  0.0627,  0.0173,  0.0527,  0.0402, -0.0720,  0.0639, -0.0492,
          0.0640, -0.0485, -0.0953,  0.0530,  0.0525,  0.0348,  0.0398,  0.0923,
         -0.0609,  0.0202,  0.0518, -0.